In [8]:
import pandas as pd
import random
from tqdm import tqdm

In [431]:
df = pd.read_csv('./data/web-Google.txt', sep='\t', lineterminator='\r')
# df = pd.read_csv('./data/test.txt', sep='\t')
# df = pd.read_csv('./data/test2.txt', sep=' ')
df

,FromNodeId,ToNodeId
0,0,11342
1,0,824020
2,0,867923
3,0,891835
4,11342,0
...,...,...
5105034,916425,547323
5105035,916425,604833
5105036,916425,632916
5105037,916425,637936


## "Egen"

In [117]:
import random
from operator import add, sub

class TriestBase:
    def __init__(self, M):
        self.M = M
        self.t = 0
        self.S = set()
        self.globalT = 0
        self.localT = {}
        self.adjecencyDict = {}

    def sampleEdge(self, u, v):
        if self.t <= self.M:
            return True
        elif self.flipBiasedCoin():
            u_prime, v_prime = self.removeRandomEdge()
            # print('Sampled:', u_prime, v_prime)
            self.updateCounters(u_prime, v_prime, sub)
            return True
        return False

    def updateCounters(self, u, v, operator):
        common_neighborhood = self.getNeighborhoodIntersection(u,v)
        
        if not common_neighborhood:
            return

        uNotPopped = True
        vNotPopped = True

        for c in common_neighborhood:
            if operator == add:
                self.globalT += 1
                self.localT[c] = self.localT[c] + 1 if c in self.localT else 1
                self.localT[u] = self.localT[u] + 1 if u in self.localT else 1
                self.localT[v] = self.localT[v] + 1 if v in self.localT else 1

            elif operator == sub:
                # print('common_neighborhood', common_neighborhood)
                self.globalT -= 1
                try:
                    self.localT[c] -= 1
                except:
                    cInS = [edge for edge in self.S if c in edge]
                    raise Exception('C in S:', cInS)
                if uNotPopped:
                    self.localT[u] -= 1
                if vNotPopped:
                    self.localT[v] -= 1

                edgesToRemove = set()

                if self.localT[c] == 0:
                    self.localT.pop(c)
                    print('Popped:', c)
                    
                    # Remove c from all adjencency-sets
                    for key in self.adjecencyDict:
                        if self.adjecencyDict[key] & {c} == c:
                            self.adjecencyDict[key].remove(c)
                            edgesToRemove = edgesToRemove | {(key, vertice), (vertice, key)}
                    # Remove all edges to/from c
                    for vertice in self.adjecencyDict[c]:
                        edgesToRemove = edgesToRemove | {(c, vertice), (vertice, c)}
                    # Remove c from adjencency-dict
                    self.adjecencyDict.pop(c)

                if uNotPopped:
                    if self.localT[u] == 0:
                        self.localT.pop(u)
                        uNotPopped = False
                        print('Popped:', u)
                        
                        # Remove u from all adjencency-sets
                        for key in self.adjecencyDict:
                            if self.adjecencyDict[key] & {u} == u:
                                self.adjecencyDict[key].remove(u)
                                edgesToRemove = edgesToRemove | {(key, vertice), (vertice, key)}
                        # Remove all edges to/from u
                        for vertice in self.adjecencyDict[u]:
                            edgesToRemove = edgesToRemove | {(u, vertice), (vertice, u)}
                        # Remove v from adjencency-dict
                        self.adjecencyDict.pop(u)

                if vNotPopped:
                    if self.localT[v] == 0:
                        self.localT.pop(v)
                        vNotPopped = False
                        print('Popped:', v)
                        
                        # Remove v from all adjencency-sets
                        for key in self.adjecencyDict:
                            if self.adjecencyDict[key] & {v} == v:
                                self.adjecencyDict[key].remove(v)
                                edgesToRemove = edgesToRemove | {(key, vertice), (vertice, key)}
                        # Remove all edges to/from v
                        for vertice in self.adjecencyDict[v]:
                            edgesToRemove = edgesToRemove | {(v, vertice), (vertice, v)}
                        # Remove v from adjencency-dict
                        self.adjecencyDict.pop(v)


                if edgesToRemove:
                    print('edgesToRemove', edgesToRemove)
                    for edge in edgesToRemove:
                        self.S.discard(edge)

    def flipBiasedCoin(self):
        coinFlip = random.random()
        return True if coinFlip <= self.M/self.t else False


    def return_counters(self):
        estimate = max(1, (self.t * (self.t - 1) * (self.t - 2))/(self.M * (self.M - 1) * (self.M - 2)))

        global_estimate = int(estimate * self.globalT)

        for key in self.localT:
            self.localT[key] = int(self.localT[key] * estimate)

        return {'global':global_estimate,'local':self.localT}

    def addEdge(self, u, v):
        self.S = self.S | {(u, v)}
        
        if u in self.adjecencyDict:
            self.adjecencyDict[u] = self.adjecencyDict[u] | {v}
        else:
            self.adjecencyDict[u] = set([v])
        
        if v in self.adjecencyDict:
            self.adjecencyDict[v] = self.adjecencyDict[v] | {u}
        else:
            self.adjecencyDict[v] = set([u])

    def removeRandomEdge(self):
        edgeToRemove = random.sample(self.S, 1)[0]
        # print('Removed:', edgeToRemove)
        self.S.remove(edgeToRemove)
        u, v = list(edgeToRemove)[0], list(edgeToRemove)[1]
        self.adjecencyDict[u].discard(v)
        self.adjecencyDict[v].discard(u)
        return u, v

    def getNeighborhood(self, u):
        # N = set()
        # # print('getNeighborhood, u:', u)
        # for edge in self.S:
        #     if u in edge:
        #         for vertice in edge:
        #             N = set().union(N, {vertice})
        # # print('getNeighborhood, N before deletion:', N)
        # if N:
        #     N.remove(u)
        # # print('getNeighborhood, N after deletion:', N)
        # return N
        if u in self.adjecencyDict:
            return self.adjecencyDict[u]
        else:
            return set()

    def getNeighborhoodIntersection(self, u, v):
        uNeighbors = self.getNeighborhood(u)
        vNeighbors = self.getNeighborhood(v)
        neighborIntersection = uNeighbors & vNeighbors
        return neighborIntersection

    def run(self, u, v):
        self.t += 1
        if self.sampleEdge(u, v):
            self.addEdge(u, v)
            self.updateCounters(u, v, add)

## Ny

In [53]:
random.random()

0.2876406534994853

In [427]:
import random
# from operator import add, sub

class TriestBaseNy:
    def __init__(self, M, directed=True):
        self.M = M
        self.directed = directed
        self.t = 0
        self.S = {}
        self.globalT = 0
        self.localT = {}
        self.adds = 0
        self.sub = 0

    def flipBiasedCoin(self):
        coinFlip = random.random()
        if coinFlip <= self.M/self.t:
            return True
        else:
            return False
        # return True if coinFlip <= self.M/self.t else False
  
    def addEdge(self, u, v):
        if self.directed:
            if u not in self.S:
                self.S[u] = {'in': set(), 'out': set([v])}
            else:
                self.S[u]['out'] = self.S[u]['out'] | {v}
            if v not in self.S:
                self.S[v] = {'in': set([u]), 'out': set()}
            else:
                self.S[v]['in'] = self.S[v]['in'] | {u}
        else:
            if u not in self.S:
                self.S[u] = {'in': set([v]), 'out': set([v])}
            else:
                self.S[u]['in'] = self.S[u]['in'] | {v}
                self.S[u]['out'] = self.S[u]['out'] | {v}
            if v not in self.S:
                self.S[v] = {'in': set([u]), 'out': set([u])}
            else:
                self.S[v]['in'] = self.S[v]['in'] | {u}
                self.S[v]['out'] = self.S[v]['out'] | {u}
        

    def sampleEdge(self, u, v):
        if self.t <= self.M:
            return True
        elif self.flipBiasedCoin():
            u_prime, v_prime = self.removeRandomEdge()
            self.updateCounters(u_prime, v_prime, '-')
            return True
        return False

    def removeRandomEdge(self):
        
        # Pick a random vertice
        randomVerticeFrom = random.sample(self.S.keys(), 1)[0]
        # If that vertice has out edges
        if self.S[randomVerticeFrom]['out']:
            # Pick a random edge
            # print('First IF')
            randomVerticeTo = random.sample(self.S[randomVerticeFrom]['out'], 1)[0]
        # If the vertice does not have out edges
        else:
            # Set To = From
            randomVerticeTo = randomVerticeFrom
            # Pick a random in vertice
            # print('Second IF')
            randomVerticeFrom = random.sample(self.S[randomVerticeTo]['in'], 1)[0]

        self.cleanUp(randomVerticeFrom, randomVerticeTo)

        return randomVerticeFrom, randomVerticeTo

    def cleanUp(self, u, v):
        if self.directed:
            self.S[u]['out'].remove(v)
            self.S[v]['in'].remove(u)

        else:
            self.S[u]['in'].remove(v)
            self.S[u]['out'].remove(v)

            self.S[v]['in'].remove(u)
            self.S[v]['out'].remove(u)

        # If vertice has no edges, drop it completely
        if (len(self.S[u]['in']) + len(self.S[u]['out'])) == 0:
            self.S.pop(u)

        if (len(self.S[v]['in']) + len(self.S[v]['out'])) == 0:
            self.S.pop(v)



    def getNeighborhoodIntersection(self, u, v):
        # uNeighbors = self.S[u]['out']
        # vNeighbors = self.S[v]['in']
        neighborIntersection = {}
        if u in self.S and v in self.S:
            neighborIntersection = (self.S[u]['out'] & self.S[u]['in'])# & (self.S[v]['in'] & self.S[v]['out'])
        else:
            neighborIntersection = None
        return neighborIntersection

    def updateCounters(self, u, v, operator):
        common_neighborhood = self.getNeighborhoodIntersection(u,v)

        # print((common_neighborhood))

        if not common_neighborhood:
            return

        for c in common_neighborhood:

            if operator == '+':
                self.globalT += 1
                self.adds += 1
                
                if c in self.localT:
                    self.localT[c] += 1
                else:
                    self.localT[c] = 1

                if u in self.localT:
                    self.localT[u] += 1
                else:
                    self.localT[u] =1

                if v in self.localT:
                    self.localT[v] += 1
                else:
                    self.localT[v] = 1

            elif operator == '-':
                self.globalT -= 1
                self.sub += 1
                
                self.localT[c] -= 1

                if self.localT[c] == 0:
                    self.localT.pop(c)

                self.localT[u] -= 1
                
                if self.localT[u] == 0:
                    self.localT.pop(u)

                self.localT[v] -= 1
                if self.localT[v] == 0:
                    self.localT.pop(v)

        # for c in common_neighborhood:
        #     if operator == add:
        #         self.adds += 1
        #     else:
        #         self.sub += 1
        #     self.globalT = operator(self.globalT, 1)
        #     self.localT[c] = operator(self.localT[c], 1) if c in self.localT else 1
        #     self.localT[u] = operator(self.localT[u], 1) if u in self.localT else 1
        #     self.localT[v] = operator(self.localT[v], 1) if v in self.localT else 1

        #     if self.localT[c] < 1:
        #         self.localT.pop(c)
        #     if self.localT[u] < 1:
        #         self.localT.pop(u)
        #     if self.localT[v] < 1:
        #         self.localT.pop(v)

        # if operator == sub:
        #     self.cleanUp(u, v)

    def return_counters(self):
        estimate = max(1, (self.t * (self.t - 1) * (self.t - 2))/(self.M * (self.M - 1) * (self.M - 2)))

        print(estimate)

        global_estimate = int(estimate * self.globalT)

        for key in self.localT:
            self.localT[key] = int(self.localT[key] * estimate)

        print('adds', self.adds, 'subs', self.sub)

        return {'global':global_estimate,'local':self.localT}


    def run(self, u, v):
        self.t += 1
        if self.sampleEdge(u, v):
            self.addEdge(u, v)
            self.updateCounters(u, v, '+')

## Github

In [394]:
import random
from collections import defaultdict

class EdgeSample:
    def __init__(self):
        self._S = []
        self._neighborhood = defaultdict(set)

    def add_edge(self,u,v):
        self._S.append((u,v))
        self.edit_neighborhood('+',u,v)

    def remove_random_edge(self):
        rand_choice = random.randint(0,len(self._S)-1)
        u_dash,v_dash = self._S.pop(rand_choice)
        self.edit_neighborhood('-',u_dash,v_dash)
        return u_dash, v_dash

    """def get_intersection_neighborhood(self,u,v):
        u_neighbors = set()
        v_neighbors = set()
        for edge in self._S:
            if u in edge or v in edge:
                if u in edge:
                    u_neighbors.add(edge[edge.index(u)-1]) # if index(u)=0, picks -1 and index(1), picks 0
                if v in edge:
                    v_neighbors.add(edge[edge.index(v)-1])
 
        return u_neighbors.intersection(v_neighbors)"""

    def get_intersection_neighborhood(self,u,v):
        if u in self._neighborhood and v in self._neighborhood:
            return self._neighborhood[u].intersection(self._neighborhood[v])
        else:
            return None


    def edit_neighborhood(self,op,u,v):
        if op == '+':
            self._neighborhood[u].add(v)
            self._neighborhood[v].add(u)

        elif op == '-':
            try:
                self._neighborhood[u].remove(v)
                self._neighborhood[v].remove(u)
            except:
                pass

            if not self._neighborhood[u]:
                self._neighborhood.pop(u)

            if not self._neighborhood[v]:
                self._neighborhood.pop(v)

class TriestBaseGit:
    def __init__(self,M):
        self._M = M
        self._sample = EdgeSample()
        self._globalT = 0
        self._localT = {}
        self._t = 0
        self.add = 0
        self.sub = 0

    def sample_edge(self,u,v):
        if self._t <= self._M:
            return True
        elif self.flip_biased_coin():
            u_dash, v_dash = self._sample.remove_random_edge()
            self.update_counters(u_dash,v_dash,'-')
            return True
        return False

    def update_counters(self,u,v,op):
        common_neighborhood = self._sample.get_intersection_neighborhood(u,v)
        # print(u,v,common_neighborhood)
        # print((common_neighborhood))
        
        if not common_neighborhood:
            return

        for c in common_neighborhood:

            if op == '+':
                self._globalT += 1
                self.add += 1
                
                if c in self._localT:
                    self._localT[c] += 1
                else:
                    self._localT[c] = 1

                if u in self._localT:
                    self._localT[u] += 1
                else:
                    self._localT[u] =1

                if v in self._localT:
                    self._localT[v] += 1
                else:
                    self._localT[v] = 1

            elif op == '-':
                self._globalT -= 1
                self.sub += 1
                
                if c in self._localT:
                    self._localT[c] -= 1

                    if self._localT[c] == 0:
                        self._localT.pop(c)

                if u in self._localT:
                    self._localT[u] -= 1
                    
                    if self._localT[u] == 0:
                        self._localT.pop(u)

                if v in self._localT:
                    self._localT[v] -= 1
                    if self._localT[v] == 0:
                        self._localT.pop(v)

    def flip_biased_coin(self):
        head_prob = random.random()

        if head_prob <= self._M/self._t:
            return True
        else:
            return False

    def return_counters(self):
        estimate = max(1, (self._t * (self._t - 1) * (self._t - 2))/(self._M * (self._M - 1) * (self._M - 2)))

        print(estimate)

        global_estimate = int(estimate * self._globalT)

        for key in self._localT:
            self._localT[key] = int(self._localT[key] * estimate)
        
        print('adds', self.add, 'subs', self.sub)

        return {'global':global_estimate,'local':self._localT}

    def run(self,u,v):
        self._t += 1
        if self.sample_edge(u,v):
            self._sample.add_edge(u,v)
            self.update_counters(u,v,'+')

## Linus

In [11]:
from collections import defaultdict
import random

class Graph:
    """
    A graph object to simplify the implementation of the algorithm
    """
    def __init__(self):
        self.adj_list = defaultdict(set)
        self.edges = 0
    
    def add_edge(self, edge):
        u, v = edge
        self.adj_list[u].add(v)
        self.adj_list[v].add(u)
        self.edges = self.edges + 1
    
    def remove_edge(self, edge):
        u, v = edge
        # remove edge from all neighbors
        self.adj_list[u].remove(v)
        self.adj_list[v].remove(u)
        # the paper says to remove the vertex if it has no neighbors
        self.prune_empty_vertices(edge)
        self.edges = self.edges - 1
    
    def prune_empty_vertices(self, edge):
        u, v = edge
        # adjacency list empty -> remove vertex
        if len(self.adj_list[u]) == 0:
            del self.adj_list[u]
        if len(self.adj_list[v]) == 0:
            del self.adj_list[v]

    def neighbors(self, edge):
        u, v = edge
        # since the edge might come from the stream, we need to check if it is in the graph.
        # If it is not, we return an empty set, else we return the adjecency list of respective vertex
        if u not in self.adj_list or v not in self.adj_list:
            return set()
        else:
            return self.adj_list[u] & self.adj_list[v]
    
    def yield_random_edge(self):
        # pick a random vertex from the graph
        x = random.choice(list(self.adj_list))
        # pick a random neighbor, vertex y, of x
        y = random.choice(list(self.adj_list[x]))
        return (x, y)



import numpy as np
from random import sample
from operator import add, sub

class TriestBaseLinus:
    # set up nasty globals >_<
    def __init__(self, stream, M=6):
        self.stream = stream
        self.tau = defaultdict(int)
        # TODO borde S vara en adjecency list?
        self.S = Graph()
        self.t = 0
        self.M = M

    def run(self):
        # Here I simulate a stream of edges
        for edge in tqdm(self.stream):
            self.t+=1
            if self.sample_edge(edge):
                self.S.add_edge(edge)
                self.update_counters(operator=add, edge=edge)

    def sample_edge(self, edge):
        if self.t <= self.M:
            return True
        elif self.flip_biased_coin(self.M/self.t):
            # pick a random edge from S
            random_edge = self.S.yield_random_edge()
            # remove it from S
            self.S.remove_edge(random_edge)
            self.update_counters(operator=sub, edge=random_edge)
            return True
        else:
            return False

    def flip_biased_coin(self, p):
        # TODO is olikheten correct or should it be '>'?
        return np.random.rand() < p

    def update_counters(self, operator, edge):
        u, v = edge
        neighborhood = self.S.neighbors(edge)
        for c in neighborhood:
            # bug here, the operator is malfunctioning???
            self.tau['global'] = operator(self.tau['global'], 1)
            self.tau[c] = operator(self.tau[c], 1)
            self.tau[u] = operator(self.tau[u], 1)
            self.tau[v] = operator(self.tau[v], 1)
    
    def xi(self, t, M):
        f = lambda x: x * (x-1) * (x-2)
        candidate = f(t) / f(M)
        print(f"t: {t}, M: {M}, tau: {self.tau['global']}")
        print(f"f(t): {f(t)}, f(M): {f(M)}")
        print(f"candidate f(t)/f(M): {candidate}")
        print('global:', max(1, candidate) * self.tau['global'])
        return np.max([1.0, candidate])
    
    def global_estimate(self):
        return self.xi(self.t, self.M) * self.tau['global']


# estimate = max(1, (self._t * (self._t - 1) * (self._t - 2))/(self._M * (self._M - 1) * (self._M - 2)))

# global_estimate = int(estimate * self._globalT)

# for key in self._localT:
#     self._localT[key] = int(self._localT[key] * estimate)

# return {'global':global_estimate,'local':self._localT}

## Testing

In [429]:
M = 7000

test = TriestBaseNy(M, directed=False)
for i in tqdm(range(len(df))):
  fromNodeID = df.iloc[i]['FromNodeId']
  toNodeId = df.iloc[i]['ToNodeId']
  # print('Run Edge: (' + str(fromNodeID) + ', ' + str(toNodeId) + ')')
  test.run(fromNodeID, toNodeId)

print(len(test.S))

print(test.return_counters())

# 5 619 756 637 730   Daniel 7k
# 5 546 026 920 760   Linus 7k
#   671 060 717 990   Linus 20k
#        13 391 903   Truth
#   611 568 599 714   Daniel 7k
# 2 496 332 996 169
#   593 330 196 042

 28%|██▊       | 55487/198050 [00:08<00:21, 6737.57it/s] 


KeyError: 880

In [432]:
M = 7000

testGit = TriestBaseGit(M)
for i in tqdm(range(len(df))):
  fromNodeID = df.iloc[i]['FromNodeId']
  toNodeId = df.iloc[i]['ToNodeId']
  testGit.run(fromNodeID, toNodeId)

print(testGit.return_counters())


  0%|          | 8222/5105039 [00:00<08:57, 9482.57it/s] 


KeyError: 408707

In [17]:
M = 20000

linusStream = list(df.itertuples(index=False))
testLinus = TriestBaseLinus(linusStream, M)
testLinus.run()
testLinus.global_estimate()

# estimate = max(1, (testLinus.t * (testLinus.t - 1) * (testLinus.t - 2))/(testLinus.M * (testLinus.M - 1) * (testLinus.M - 2)))
# global_estimate = int(estimate * testLinus.tau['global'])

# for key in testLinus.tau:
#   if key == 'global':
#     continue
#   testLinus.tau[key] = int(testLinus.tau[key] * estimate)

# print({'global':global_estimate,'local':testLinus.tau})

100%|██████████| 5105039/5105039 [00:48<00:00, 104435.86it/s]

t: 5105039, M: 20000, tau: 40345
f(t): 133044503603959809834, f(M): 7998800040000
candidate f(t)/f(M): 16633057.826003587
global: 671060717990.1147


671060717990.1147

In [2]:
S = set()

# S.add(frozenset([1]))

S = set.union(S, {frozenset([1, 2, 3])})

S = set.union(S, {frozenset([1, 2, 4])})

# S.remove(frozenset([1, 2, 4]))

S

{frozenset({1, 2, 3}), frozenset({1, 2, 4})}

In [48]:
M = 100
S = set()

for t in tqdm(range(len(df))):
  fromNodeId = df.iloc[t]['FromNodeId']
  toNodeId = df.iloc[t]['ToNodeId']

  # if t % 500000 == 0:
  #   print(t)
  
  if t <= M:
    S = set.union(S, {frozenset([fromNodeId, toNodeId])})
    # if len(S) == len_S:
    #   print('Duplicate edge:', {frozenset([fromNodeId, toNodeId])})

  elif random.uniform(0, 1) < (M/t):
    # print(len(S))
    # print('Removed edge:', {frozenset([fromNodeId, toNodeId])})
    edgeToRemove = random.sample(S, 1)[0]
    # print(edgeToRemove[0])
    # print(S)
    S.remove(edgeToRemove)
    S = set.union(S, {frozenset([fromNodeId, toNodeId])})

print(len(S))


    



  0%|          | 295/5105039 [00:00<29:33, 2877.94it/s]

Removed edge: {frozenset({425667, 223236})}
Removed edge: {frozenset({223236, 521022})}
Removed edge: {frozenset({223236, 522701})}
Removed edge: {frozenset({578091, 223236})}
Removed edge: {frozenset({610457, 223236})}
Removed edge: {frozenset({614906, 223236})}
Removed edge: {frozenset({674265, 223236})}
Removed edge: {frozenset({223236, 777734})}
Removed edge: {frozenset({799858, 223236})}
Removed edge: {frozenset({223236, 822292})}
Removed edge: {frozenset({223236, 867774})}
Removed edge: {frozenset({907505, 223236})}
Removed edge: {frozenset({223236, 912757})}
Removed edge: {frozenset({276233, 1})}
Removed edge: {frozenset({276233, 53051})}
Removed edge: {frozenset({276233, 164684})}
Removed edge: {frozenset({276233, 203402})}
Removed edge: {frozenset({276233, 208814})}
Removed edge: {frozenset({276233, 223236})}
Removed edge: {frozenset({276233, 240041})}
Removed edge: {frozenset({276233, 303045})}
Removed edge: {frozenset({276233, 408707})}
Removed edge: {frozenset({276233, 4361

  0%|          | 1048/5105039 [00:00<23:54, 3557.08it/s]

 {frozenset({19576, 358176})}
Removed edge: {frozenset({19576, 487767})}
Removed edge: {frozenset({226374, 43343})}
Removed edge: {frozenset({57818, 226374})}
Removed edge: {frozenset({358176, 57818})}
Removed edge: {frozenset({57818, 487767})}
Removed edge: {frozenset({226374, 43911})}
Removed edge: {frozenset({160614, 226374})}
Removed edge: {frozenset({305428, 226374})}
Removed edge: {frozenset({621443, 226374})}
Removed edge: {frozenset({780880, 226374})}
Removed edge: {frozenset({801318, 226374})}
Removed edge: {frozenset({230497, 226374})}
Removed edge: {frozenset({358176, 19576})}
Removed edge: {frozenset({358176, 338360})}
Removed edge: {frozenset({358176, 487767})}
Removed edge: {frozenset({19576, 514170})}
Removed edge: {frozenset({514170, 487767})}
Removed edge: {frozenset({877041, 514170})}
Removed edge: {frozenset({694921, 57818})}
Removed edge: {frozenset({743269, 359790})}
Removed edge: {frozenset({824952, 743269})}
Removed edge: {frozenset({666882, 55948})}
Removed edge

  0%|          | 1760/5105039 [00:00<27:10, 3129.46it/s]

...

  1%|          | 39538/5105039 [00:05<12:30, 6751.28it/s]

Removed edge: {frozenset({492824, 896318})}
Removed edge: {frozenset({807888, 309846})}
Removed edge: {frozenset({55188, 501463})}
Removed edge: {frozenset({656001, 496004})}


  1%|          | 40901/5105039 [00:05<12:00, 7028.09it/s]

Removed edge: {frozenset({181922, 567756})}
Removed edge: {frozenset({100643, 799308})}


KeyboardInterrupt: 

In [ ]:
class Graph:

  class Node:

    def __init__(self, nodeID):
      self.nodeID = nodeID
      self.neighbours = []

    def addNeighbour(self, node):
      self.neighbours.append(node)